In [7]:
import pprint
import re
from copy import deepcopy

import pandas as pd

df = pd.read_excel("./reglamentos.xlsx")

df = df.rename(
    columns={
        "REGLAMENTO GENERAL DE LA CONVENCIÓN": "titulo",
        "Unnamed: 1": "parrafos",
        "Unnamed: 2": "articulo",
        "Unnamed: 3": "texto",
        "Unnamed: 4": "lista",
        "Unnamed: 5": "sublista",
        "Unnamed: 6": "subsublista",
    }
)
df = df.iloc[:, 0:7]


In [8]:
list_index_pattern = re.compile(
    r"\D\d*\)|\d{1,3}\.|I\.|II\.|III\.|IV\.|V\.|VI\.|VII\.|VIII.\|IX\.|X\."
)


datos = []
ultimo_visto = None
lista = []

for idx, row in df.iterrows():

    try:

        # --------------------------------
        # Titulo

        if not pd.isna(row["titulo"]):
            titulo_actual = {
                "tipo": 'titulo',
                "titulo": row["titulo"],
                "descripcion": row["parrafos"],
                "contenido": [],
            }
            datos.append(titulo_actual)
            # ultimo_visto = "titulo"

        # --------------------------------
        # Parrafo
        elif not pd.isna(row["parrafos"]):
            parrafo_actual = {
                'tipo': 'parrafo',
                "titulo": row["parrafos"],
                "descripcion": row["articulo"],
                "contenido": [],
            }
            titulo_actual["contenido"].append(parrafo_actual)
            # ultimo_visto = "parrafo"

        # --------------------------------
        # Artículo

        elif not pd.isna(row["articulo"]):
            articulo_actual = {
                'tipo': 'articulo',
                "titulo": row["articulo"],
                "descripcion": row["texto"],
                "contenido": [],
            }

            parrafo_actual["contenido"].append(articulo_actual)
            # ultimo_visto = "articulo"

        # --------------------------------
        # Contenido de un Artículo
        elif not pd.isna(row["texto"]):

            if re.match(list_index_pattern, row["texto"]):
                # print(row['texto'])
                indice = row["texto"]
                articulo_actual["contenido"].append(
                    {"tipo": "lista", "indice": indice, "contenido": [row["lista"]]}
                )
                ultimo = "inicio_lista"
            else:
                texto = row["texto"]
                articulo_actual["contenido"].append(
                    {"tipo": "texto", "contenido": row["texto"]}
                )
                ultimo = "texto"

        elif not pd.isna(row["lista"]):

            if ultimo in ["inicio_lista", "punto_lista"]:
                articulo_actual["contenido"][-1]["contenido"].append(row["lista"])
                ultimo = "punto_lista"

        else: 
            # Casos no resueltos!!!!
            print('-'*60)
            print(row)

        # print("-" * 69 + f"\nidx={idx} | ultimo_visto={ultimo_visto}\n{row}")

    except Exception as e:
        print("!!" + "-" * 60 + f"\nidx={idx}\n{row}\n{locals().keys()}")
        raise Exception("ERROR!!!")

pprint.pprint(titulo_actual)

import json

with open("general.json", "w") as f:
    json.dump(datos, f, ensure_ascii=False)


------------------------------------------------------------
titulo         NaN
parrafos       NaN
articulo       NaN
texto          NaN
lista          NaN
sublista       NaN
subsublista    NaN
Name: 320, dtype: object
------------------------------------------------------------
titulo         NaN
parrafos       NaN
articulo       NaN
texto          NaN
lista          NaN
sublista       NaN
subsublista    NaN
Name: 795, dtype: object
{'contenido': [],
 'descripcion': 'Disposiciones transitorias',
 'tipo': 'titulo',
 'titulo': 'Disposiciones transitorias'}
